In [ ]:
from datasets import load_dataset,DatasetDict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import io
import os
from PIL import Image
import random
import torchvision.transforms as transforms
import utils
import torch.nn as nn
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
import time

In [ ]:
ds = load_dataset("Karthik11232/human_face_generation")
#split the dataset 
train_test_set = ds['train'].train_test_split(test_size=0.2)
#print(train_test_set)
train_val_set = train_test_set['train'].train_test_split(test_size=0.2)

ds = DatasetDict({
    'train': train_test_set['train'],
    'test': train_test_set['test'],
    'validation': train_val_set['test']
})
#print (ds)
print(ds.shape)
print(train_test_set.shape)
print(train_val_set.shape)

In [ ]:
# Save the datasets to disk
output_dir = './human_face_generation_split/raw'
if not os.path.exists(output_dir):  
    os.makedirs(output_dir, exist_ok=True)
# Save each split to disk
    utils.save_images(ds['train'], 'train',output_dir)
    utils.save_images(ds['validation'], 'validation')
    utils.save_images(ds['test'], 'test')

In [ ]:
# Plot 15 random images in 3 rows and 5 columns
utils.plot_random_images(dataset_path='human_face_generation_split/raw/train', num_images=15, rows=3, cols=5)

In [ ]:
# Prepoccessing 
preprocess = transforms.Compose([
    transforms.Resize((200, 200)),  # Resize to a specific size if needed
    transforms.ToTensor(),          # Convert to tensor and normalize
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
preprocessed_output_dir = './human_face_generation_split/preprocessed'
os.makedirs(preprocessed_output_dir, exist_ok=True)
if not os.path.exists(preprocessed_output_dir):
# Save preprocessed training images

    utils.save_images(ds['train'], 'train', preprocessed_output_dir,preprocess=preprocess)
    utils.plot_random_images(os.path.join(preprocessed_output_dir, 'train'), num_images=15, rows=3, cols=5)
    img = Image.open("human_face_generation_split/preprocessed/train/image_2110.png")
    print(img.size)


In [ ]:
# first implement a simple Autoencoder with Linear layers
class Autoencoder_linear(nn.Module):
    def __init__(self):
        super().__init__()
        self.size_hidden_layers = 512 
        self.encoder = nn.Sequential(
            nn.Linear(40000,1000),
            nn.ReLU(),
            nn.Linear(1000,self.size_hidden_layers),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.size_hidden_layers,1000),
            nn.ReLU(),
            nn.Linear(1000,40000),
            nn.ReLU(),
        )
    def init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.01)
   # init_weights(self.encoder)
    def forward(self,x):
        encoded=self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded,decoded

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self,image_folder):
        self.image_folder=image_folder
        self.images = os.listdir (image_folder)
        self.transform = transforms.Compose([
            transforms.Resize((200, 200)),  # Resize images to 200x200
            transforms.ToTensor()           # Convert images to PyTorch tensors
        ])
    def __getitem__(self,idx):
        image_file = self.images[idx]
        image = Image.open((self.image_folder +"/"+ image_file))
        image = self.transform(image)
        #image = np.array(image).reshape(-1,200*200) for lin autoencoder


        image = torch.Tensor(image)
       # print(image.shape)

       # print (image.shape)
        return image
    def __len__(self):
        return len(self.images)
    
dataset_preprocessed = ImageDataset("human_face_generation_split/preprocessed/train")
dataset_val = ImageDataset("human_face_generation_split/raw/validation")
dataset_test = ImageDataset("human_face_generation_split/raw/test")
show_dataset = ImageDataset("human_face_generation_split/show_imgs/train")

train_data = torch.utils.data.DataLoader(dataset_preprocessed,batch_size=64,shuffle=True)
val_data = torch.utils.data.DataLoader(dataset_val)
test_data = torch.utils.data.DataLoader(dataset_test)
show_data =torch.utils.data.DataLoader(show_dataset)

In [ ]:
#ckp_path="human_face_generation_split_data{}.ckp"
crt_time = time.strftime("%Y%m%d_%H%M%S")
dir_ckpt = os.path.join("human_face_generation_split/",str(crt_time))
os.mkdir(dir_ckpt)
writer = SummaryWriter()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
linear_auto = Autoencoder_linear()
print(linear_auto)
linear_auto.to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(linear_auto.parameters(), lr=1e-4)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

epochs =  100
#training
# Training
for epoch in range(epochs):
    total_loss = 0
    with tqdm(train_data, unit="batch", total=len(train_data), desc=f"Epoch {epoch+1}/{epochs}") as tepoch:
        for batch_idx, data in enumerate(tepoch):
            data = data.to(device)
            encoded, decoded = linear_auto(data)
            loss = loss_function(decoded, data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * data.size(0)
            
            # Set the postfix with current loss
            tepoch.set_postfix(loss=total_loss / ((batch_idx + 1) * data.size(0)))
            writer.add_scalar("Loss/train", loss, epoch)

            #print(decoded.shape)

    if epoch % 2 == 0:
        linear_auto.eval()  # Set the model to evaluation mode
        ckpt_file = os.path.join(dir_ckpt, f"checkpoint_epoch_{epoch+1}.pt")

        torch.save({
            'epoch': epoch,
            'state_dict': linear_auto.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss,
            
        },ckpt_file)
    epoch_loss = total_loss / len(train_data.dataset)
    print(f"Epoch {epoch+1}/{epochs} : loss = {epoch_loss:.4f}")

In [ ]:
class Autoencoder_conv(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels = 3 , out_channels = 32, kernel_size = 2 ,stride = 2),
            nn.ReLU(),# possibly add max pooling
            nn.Conv2d(in_channels = 32,out_channels = 64, kernel_size = 2, stride = 2),
            nn.ReLU()
        )    
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 64,out_channels=32,kernel_size=2,stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels = 32 , out_channels = 3 ,kernel_size=2,stride =2),
            nn.ReLU()
        )
    def forward(self,x):
            encoded = self.encoder(x)
            decoded = self.decoder(encoded)
            return encoded, decoded

In [ ]:

writer = SummaryWriter()

crt_time = time.strftime("%Y%m%d_%H%M%S")
dir_ckpt = os.path.join("human_face_generation_split/",str(crt_time)+"conv")
os.mkdir(dir_ckpt)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
autoencoder_conv = Autoencoder_conv().to(device)  # Move the model to GPU
print(autoencoder_conv)
#linear_auto = Autoencoder_linear()

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder_conv.parameters(), lr=1e-4)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

epochs =  100
#training
# Training
for epoch in range(epochs):
    total_loss = 0
    with tqdm(train_data, unit="batch", total=len(train_data), desc=f"Epoch {epoch+1}/{epochs}") as tepoch:
        for batch_idx, data in enumerate(tepoch):
            #print(data.shape)  # Add this to debug the shape

            data = data.to(device)
            encoded, decoded = autoencoder_conv(data)
            loss = loss_function(decoded, data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * data.size(0)
            
            # Set the postfix with current loss
            tepoch.set_postfix(loss=total_loss / ((batch_idx + 1) * data.size(0)))
            writer.add_scalar("Loss/train", loss, epoch)

            #print(decoded.shape)

    if (epoch+1) % 10 == 0:
        autoencoder_conv.eval()  # Set the model to evaluation mode
        ckpt_file = os.path.join(dir_ckpt, f"checkpoint_epoch_{epoch+1}.pt")

        torch.save({
            'epoch': epoch,
            'state_dict': autoencoder_conv.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss,
            
        },ckpt_file)
    epoch_loss = total_loss / len(train_data.dataset)
    print(f"Epoch {epoch+1}/{epochs} : loss = {epoch_loss:.4f}")
writer.close()

In [ ]:
#Autoencoder VAE
class AutoencoderVAE:
    def __init__(self):
        super().__init__()
    
    def encoder(self, data):
        # Define the encoder architecture
        # The output shape should be (batch_size, latent_dim, 4, 4)
        pass
    def decoder(self, data):
        # Define the decoder architecture
        # The output shape should be (batch_size, 3, 200, 200)
        pass
    def reparametrize(self, data):
        # Implement reparameterization trick to sample z from the latent space
        pass
    def forward(self, data):
        # Implement the forward pass through the VAE
        # Return the reconstructed data, the latent mean, and the latent log variance
        pass
